# Sentiment analysis using Transformer

## Import necessary libs

In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
!pip install transformers datasets huggingface_hub evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.3 MB/s eta 0:00:00


## Load and process data

In [4]:
import pandas as pd

imdb_vi = pd.read_csv('/kaggle/input/vi-imdb/VI_IMDB.csv')

In [5]:
imdb_vi.head()

,review,sentiment,vi_review
0,One of the other reviewers has mentioned that ...,positive,Một trong những người đánh giá khác đã đề cập ...
1,A wonderful little production. <br /><br />The...,positive,Một sản phẩm nhỏ tuyệt vời. <br /><br />Kỹ thu...
2,I thought this was a wonderful way to spend ti...,positive,Tôi nghĩ đây là một cách tuyệt vời để dành thờ...
3,Basically there's a family where a little boy ...,negative,"Về cơ bản, có một gia đình mà một cậu bé (Jake..."
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,"""Love in the Time of Money"" của Petter Mattei ..."


In [6]:
imdb_vi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
 2   vi_review  50000 non-null  object
dtypes: object(3)
memory usage: 1.1+ MB


In [7]:
imdb_vi.drop('review', axis=1, inplace=True)

In [8]:
imdb_vi.loc[imdb_vi[imdb_vi['sentiment'] == 'positive'].index, 'label'] = 1
imdb_vi.loc[imdb_vi[imdb_vi['sentiment'] == 'negative'].index, 'label'] = 0

In [9]:
imdb_vi.drop('sentiment', axis=1, inplace=True)

In [10]:
imdb_vi.columns = ['text', 'label']

In [11]:
imdb_vi.head()

,text,label
0,Một trong những người đánh giá khác đã đề cập ...,1.0
1,Một sản phẩm nhỏ tuyệt vời. <br /><br />Kỹ thu...,1.0
2,Tôi nghĩ đây là một cách tuyệt vời để dành thờ...,1.0
3,"Về cơ bản, có một gia đình mà một cậu bé (Jake...",0.0
4,"""Love in the Time of Money"" của Petter Mattei ...",1.0


In [12]:
data_imdb = [{'text': imdb_vi['text'].iloc[i], 'label': int(imdb_vi['label'].iloc[i])} for i in range(len(imdb_vi))]

In [13]:
data_imdb[0]

{'text': 'Một trong những người đánh giá khác đã đề cập rằng sau khi xem chỉ 1 tập Oz, bạn sẽ bị cuốn hút. Họ nói đúng, vì đây chính xác là những gì đã xảy ra với tôi.<br /><br />Điều đầu tiên khiến tôi ấn tượng về Oz là sự tàn bạo và những cảnh bạo lực không hề nao núng, thể hiện ngay từ chữ ĐI. Tin tôi đi, đây không phải là một chương trình dành cho những người yếu tim hay nhút nhát. Chương trình này không có cú đấm nào liên quan đến ma túy, tình dục hay bạo lực. Nó là hạng nặng, theo cách sử dụng cổ điển của từ này.<br /><br />Nó được gọi là OZ vì đó là biệt danh được đặt cho Trại giam Nhà nước An ninh Tối đa Oswald. Nó tập trung chủ yếu vào Thành phố Ngọc lục bảo, một khu vực thử nghiệm của nhà tù nơi tất cả các phòng giam đều có mặt trước bằng kính và hướng vào trong, vì vậy quyền riêng tư không được đề cao trong chương trình nghị sự. Thành phố Em là nơi sinh sống của nhiều người..Người Aryan, người Hồi giáo, xã hội đen, người Latinh, người theo đạo Cơ đốc, người Ý, người Ireland,

In [14]:
from sklearn.model_selection import train_test_split

small_train_imdb, small_test_imdb = train_test_split(data_imdb, test_size=0.2, random_state=42)

In [15]:
from datasets import Dataset

small_train_dataset_imdb = Dataset.from_list(small_train_imdb)
small_test_dataset_imdb = Dataset.from_list(small_test_imdb)

## Fine-tuning DistilBERT

### Tokenization

In [16]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [17]:
def preprocess_function(examples):
   return tokenizer(examples["text"], truncation=True)

tokenized_train = small_train_dataset_imdb.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset_imdb.map(preprocess_function, batched=True)

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [19]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

### Call pre-trained model

In [20]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
import numpy as np
from evaluate import load

def compute_metrics(eval_pred):
   load_accuracy = load("accuracy")
   load_f1 = load("f1")

   logits, labels = eval_pred
   logits = np.array(logits)
   predictions = np.argmax(logits, axis=-1)
   accuracy = load_accuracy.compute(predictions=predictions, references=labels)["accuracy"]
   f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
   return {"accuracy": accuracy, "f1": f1}

In [22]:
from huggingface_hub import notebook_login
notebook_login()

### Configure hyperparameters and fine-tune

In [23]:
from transformers import TrainingArguments, Trainer

repo_name = "finetuning-sentiment-model-distilBERT"

training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=5,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=True,
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

/tmp/ipykernel_23/3784610196.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [24]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
500,0.559400
1000,0.453200
1500,0.401100
2000,0.359800
2500,0.334400
3000,0.283800
3500,0.272400
4000,0.244100
4500,0.215400
5000,0.214800


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0

TrainOutput(global_step=6250, training_loss=0.3020091552734375, metrics={'train_runtime': 5543.0235, 'train_samples_per_second': 36.081, 'train_steps_per_second': 1.128, 'total_flos': 2.64934797312e+16, 'train_loss': 0.3020091552734375, 'epoch': 5.0})

In [25]:
trainer.evaluate()

/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.36198890209198,
 'eval_accuracy': 0.8703,
 'eval_f1': 0.8728057271746593,
 'eval_runtime': 94.4065,
 'eval_samples_per_second': 105.925,
 'eval_steps_per_second': 3.315,
 'epoch': 5.0}

In [39]:
print(model.config)

DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.46.3",
  "vocab_size": 30522
}



In [26]:
trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1734070931.3112e3711c46.23.0:   0%|          | 0.00/7.88k [00:00<?, ?B/s]

events.out.tfevents.1734076584.3112e3711c46.23.1:   0%|          | 0.00/457 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/takanami12/finetuning-sentiment-model-distilBERT/commit/9270b26a7a1f281b7f2f7c8b2ab4b0fce2a79653', commit_message='End of training', commit_description='', oid='9270b26a7a1f281b7f2f7c8b2ab4b0fce2a79653', pr_url=None, repo_url=RepoUrl('https://huggingface.co/takanami12/finetuning-sentiment-model-distilBERT', endpoint='https://huggingface.co', repo_type='model', repo_id='takanami12/finetuning-sentiment-model-distilBERT'), pr_revision=None, pr_num=None)

In [38]:
from transformers import pipeline

sentiment_model = pipeline(model="takanami12/finetuning-sentiment-model-distilBERT")
sentiment_model(["Bộ phim này dở quá", "Bộ phim này thật xuất sắc", "Bộ phim chán ngắt", "Bộ phim này hay voãi đạn"])

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'LABEL_0', 'score': 0.92476487159729},
 {'label': 'LABEL_1', 'score': 0.9926365613937378},
 {'label': 'LABEL_0', 'score': 0.6944849491119385},
 {'label': 'LABEL_1', 'score': 0.9899213314056396}]